# EDA: Digging into the Frequency and Severity of the dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [22]:
motor_df = pd.read_csv('../data/input/exp/Motor vehicle insurance data.csv', delimiter=";")
sample_df = pd.read_csv('../data/input/exp/sample type claim.csv', delimiter=';')

### Claims Frequency Distribution

- Number: Categorical group for number of claims ranging from 0 to 9 (maximum number of claims made by 1 policyholder)
- Policies: Number of unique policies per group
- Claims: Total number of claims per group
- Proportion: Ratio of each group policies to total number of policies.

In [65]:
combined  =  pd.merge(motor_df, sample_df, on='ID', how='left')
bins = [0,1,2,3,4,5,6,7,8,9]
labels = ['0','1','2','3','4','5','6','7','8+']
claims_frequency_summary = combined.groupby(
    pd.cut(combined['N_claims_year'], bins=bins, labels=labels), observed=False).agg(
        {
            'ID': 'nunique',
            'N_claims_year': 'sum'
        }
    ).rename(columns={
        'ID': 'Policies',
        'N_claims_year': 'Claims'
    }).reset_index().rename(columns={
        'N_claims_year': 'Number'
    })

claims_frequency_summary['proportion'] =   (claims_frequency_summary['Policies'] / claims_frequency_summary['Policies'].sum()).round(4)
claims_frequency_summary.loc[len(claims_frequency_summary)] = ['Sum', 
                                                               claims_frequency_summary['Policies'].sum(),
                                                               claims_frequency_summary['Claims'].sum(), 
                                                               claims_frequency_summary['proportion'].sum()] 
claims_frequency_summary = claims_frequency_summary.T
claims_frequency_summary.columns =  [''] * len(claims_frequency_summary.columns)
claims_frequency_summary.index.name = ''
claims_frequency_summary


,,,,,,,,,,
,,,,,,,,,,
Number,0,1,2,3,4,5,6,7,8+,Sum
Policies,9464,4952,2434,1188,609,318,227,136,82,19410
Claims,9551,12102,9210,5684,3525,2118,1631,1144,882,45847
proportion,0.4876,0.2551,0.1254,0.0612,0.0314,0.0164,0.0117,0.007,0.0042,1.0
